In [1]:
import os
from dotenv import load_dotenv
from typing import Any, Optional

from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator
from langchain.smith import EvaluationConfig, run_on_dataset

# --- Setup ---
load_dotenv()
client = Client()

# --- My Custom Tweak: An Experiment to Compare Two Models ---
print("--- Starting Experiment: GPT-3.5-Turbo vs. GPT-4o ---")
experiment_project_name = "Experiment: Summarizer Model Comparison"

def is_concise(run, example) -> dict:
    llm_output = run.outputs.get("text")
    if llm_output is None: return {"key": "is_concise", "score": 0}
    return {"key": "is_concise", "score": int(len(llm_output) < 75)}

custom_evaluator = LangChainStringEvaluator([is_concise])
eval_config = EvaluationConfig(evaluators=[custom_evaluator])
summary_prompt = PromptTemplate.from_template("Summarize the following article in one sentence: {article}")

# --- Test Run 1: Using GPT-3.5-Turbo ---
print("Running evaluation for gpt-3.5-turbo...")
summary_chain_gpt3 = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0), prompt=summary_prompt)
run_on_dataset(
    client=client,
    dataset_name="Article Summaries",
    llm_or_chain_factory=summary_chain_gpt3,
    evaluation=eval_config,
    project_name=experiment_project_name,
)
print("Finished evaluation for gpt-3.5-turbo.")

# --- Test Run 2: Using GPT-4o ---
print("\nRunning evaluation for gpt-4o...")
summary_chain_gpt4 = LLMChain(llm=ChatOpenAI(model="gpt-4o", temperature=0), prompt=summary_prompt)
run_on_dataset(
    client=client,
    dataset_name="Article Summaries",
    llm_or_chain_factory=summary_chain_gpt4,
    evaluation=eval_config,
    project_name=experiment_project_name,
)
print("Finished evaluation for gpt-4o.")
print(f"\nExperiment complete! View results in project: '{experiment_project_name}'")

ImportError: cannot import name 'EvaluationConfig' from 'langchain.smith' (c:\Users\dell\Documents\CODES\LangSmith\ls-academy\Lib\site-packages\langchain\smith\__init__.py)